
The Traffic Light Controlled Intersection problem on LeetCode requires you to simulate the operation of traffic lights at an intersection where multiple cars can pass based on certain constraints.

Problem Description
You are given three functions for cars coming from three different directions (East, North, and West), and these cars must pass through an intersection that is controlled by a traffic light. The traffic light allows only one direction to pass at a time.

Each function (carArrives) represents the arrival of a car at the intersection. If the traffic light is green for the direction from which the car is coming, the car can pass; otherwise, the car must wait until the light turns green.

Your task is to implement the simulation of this intersection, making sure cars pass in the correct order based on the traffic light rules.

Approach
The key here is to use synchronization to make sure only one direction's cars can pass at a time, as well as ensuring that cars are only allowed to pass when their traffic light is green.

A mutex lock or other thread synchronization mechanisms can be used to coordinate the passing of cars from different directions.

Steps:
Lock Mechanism: Use a lock to ensure only one car passes at a time.
Green Light Condition: Maintain a "green light" state that indicates which direction is allowed to pass.
Waiting: If a car arrives when the light is not green for its direction, it must wait.
Change of Light: After a car passes, change the light for the next set of cars.

In [2]:
import threading
from time import sleep

class TrafficLight:
    def __init__(self):
        # Initially, green light is for road 1 (East)
        self.green_light = 1
        self.lock = threading.Lock()

    def carArrived(self,
                   carId: int,           # ID of the car
                   roadId: int,          # ID of the road the car travels on. 1: East, 2: North, 3: West
                   direction: str,       # 'E' for east, 'N' for north, 'W' for west
                   turnGreen: 'Callable[[], None]',  # Use this function to turn light to green
                   crossCar: 'Callable[[], None]'):  # Use this function to make the car cross the intersection
        # Synchronize the access to the intersection
        with self.lock:
            # If the light is not green for the current road, turn it green
            if self.green_light != roadId:
                print(f"Traffic light turns green on road {roadId} for car {carId} from {direction}")
                turnGreen()
                self.green_light = roadId
            # Let the car pass through the intersection
            print(f"Car {carId} from {direction} is crossing the intersection")
            crossCar()

# Simulated car passing function
def crossCar():
    sleep(0.5)  # Simulate car crossing
    print("Car has crossed the intersection")

# Simulated light switching function
def turnGreen():
    sleep(0.1)  # Simulate time to switch the light

# Instantiate TrafficLight object
traffic_light = TrafficLight()

# Simulate cars arriving from different directions
cars = [
    (1, 1, 'East'),
    (2, 2, 'North'),
    (3, 1, 'East'),
    (4, 3, 'West'),
    (5, 2, 'North')
]

# Simulating each car arriving in separate threads
threads = []
for car in cars:
    carId, roadId, direction = car
    thread = threading.Thread(target=traffic_light.carArrived, args=(carId, roadId, direction, turnGreen, crossCar))
    threads.append(thread)
    thread.start()

# Wait for all threads to complete
for thread in threads:
    thread.join()


Car 1 from East is crossing the intersection
Car has crossed the intersection
Traffic light turns green on road 2 for car 2 from North
Car 2 from North is crossing the intersection
Car has crossed the intersection
Traffic light turns green on road 1 for car 3 from East
Car 3 from East is crossing the intersection
Car has crossed the intersection
Traffic light turns green on road 3 for car 4 from West
Car 4 from West is crossing the intersection
Car has crossed the intersection
Traffic light turns green on road 2 for car 5 from North
Car 5 from North is crossing the intersection
Car has crossed the intersection


Explanation:
Initialization: Each philosopher has access to two forks (left and right), and the self.forks array holds a lock for each fork.

Even and Odd Numbered Philosophers:

Philosophers with even indices will pick up the left fork first and then the right fork.
Philosophers with odd indices will pick up the right fork first and then the left fork.
This alternating pattern helps to avoid deadlock because it prevents a circular wait where every philosopher waits for another to release a fork.
Locks (with self.forks[i]): The with statement ensures that each philosopher can only pick up one fork at a time. After eating, they release the forks.

Time Complexity:
O(1): Since each philosopher just needs to acquire two locks (forks), eat, and release the locks.
Space Complexity:
O(1): We are using a fixed number of locks (one for each fork) and no extra space that scales with input size.

4

In [3]:
import threading
from time import sleep

class DiningPhilosophers:
    def __init__(self):
        # Each chopstick is a lock
        self.forks = [threading.Lock() for _ in range(5)]

    def wantsToEat(self,
                   philosopher: int,
                   pickLeftFork: 'Callable[[], None]',
                   pickRightFork: 'Callable[[], None]',
                   eat: 'Callable[[], None]',
                   putLeftFork: 'Callable[[], None]',
                   putRightFork: 'Callable[[], None]') -> None:
        left_fork = philosopher
        right_fork = (philosopher + 1) % 5

        if philosopher % 2 == 0:
            # Even-numbered philosophers pick up the left fork first
            with self.forks[left_fork]:
                with self.forks[right_fork]:
                    pickLeftFork()
                    pickRightFork()
                    eat()
                    putLeftFork()
                    putRightFork()
        else:
            # Odd-numbered philosophers pick up the right fork first
            with self.forks[right_fork]:
                with self.forks[left_fork]:
                    pickRightFork()
                    pickLeftFork()
                    eat()
                    putRightFork()
                    putLeftFork()

def pickLeftFork():
    print("Picked up left fork")

def pickRightFork():
    print("Picked up right fork")

def eat():
    print("Eating")
    sleep(1)  # Simulating the eating process

def putLeftFork():
    print("Put down left fork")

def putRightFork():
    print("Put down right fork")

# Instantiate the dining philosophers problem
dining_philosophers = DiningPhilosophers()

# Simulate philosophers trying to eat
def philosopher_dine(philosopher_id):
    dining_philosophers.wantsToEat(philosopher_id, pickLeftFork, pickRightFork, eat, putLeftFork, putRightFork)

# Create threads for each philosopher
threads = []
for i in range(5):
    thread = threading.Thread(target=philosopher_dine, args=(i,))
    threads.append(thread)
    thread.start()

# Wait for all threads to finish
for thread in threads:
    thread.join()


Picked up left fork
Picked up right fork
Eating
Picked up right fork
Picked up left fork
Eating
Put down right forkPut down left fork
Put down right fork
Picked up right fork
Picked up left fork
Eating

Put down left fork
Picked up left fork
Picked up right fork
Eating
Put down left fork
Put down right fork
Put down right fork
Put down left fork
Picked up left fork
Picked up right fork
Eating
Put down left fork
Put down right fork
